In [2]:
!pip install -q -U pypdf


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install tiktoken

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [402]:
import os
import tiktoken
from dotenv import load_dotenv, find_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain,LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import create_history_aware_retriever, RetrievalQA

In [387]:
load_dotenv('.env')

True

In [388]:
#model_name = sayan-gpt35trubo16k
api_key=os.environ["AZURE_OPENAI_API_KEY"]
azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
api_version=os.environ["OPENAI_API_VERSION"]
pine_cone_Api = os.environ["PINECONE_API_KEY"]
#os.environ["HUGGING_FACE_API"] = 

In [389]:
import os
from langchain.document_loaders import PyPDFLoader

# Path to the directory containing the PDF files
path = "./data/data"

# Load all PDFs and extract text
documents = []
for pdf_file in os.listdir(path):
    if pdf_file.endswith('.pdf'):
        loader = PyPDFLoader(os.path.join(path, pdf_file))
        print(f"Loading: {pdf_file}")
        documents.extend(loader.load())


Loading: genai.pdf
Loading: gpt.pdf
Loading: In_context_learning.pdf
Loading: RAG.pdf
Loading: transformers.pdf


In [390]:
len(documents)

150

In [391]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken

# Initialize the tokenizer for the specific model
encoding = tiktoken.encoding_for_model("text-embedding-ada-002")

# Initialize the RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Set desired max chunk size in tokens
    chunk_overlap=50,  # Set desired overlap in tokens between chunks
    separators=["\n\n", "\n", " ", ""],  # Split preferentially at paragraphs, then new lines, etc.
    length_function=lambda text: len(encoding.encode(text))  # Use token length
)

# Assuming `documents` contains text content from 5 PDFs
chunks = []
for document in documents:
    chunks.extend(text_splitter.split_text(document.page_content))


In [392]:
len(chunks)

223

In [393]:
# Initialize the embedding model
embed_model = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    deployment="sayan-textembeddingada002",
    api_key=api_key,
    azure_endpoint=azure_endpoint[0],
    api_version=api_version
)

In [394]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec


In [398]:
pc = Pinecone(api_key=pine_cone_Api)

index_name = 'llm-bot-2'
existing_indexes = [i.get('name') for i in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )

# connect to the index
pinecone_index = pc.Index(index_name)


In [399]:
pinecone_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 918}},
 'total_vector_count': 918}

In [177]:
max_retries = 5
for i, chunk in enumerate(chunks):
    retries = 0
    while retries < max_retries:
        try:
            # Process each chunk
            PineconeVectorStore.from_texts(
                texts=[chunk],  # Process one chunk at a time
                index_name=index_name,
                embedding=embed_model
            )
            print(f"Successfully processed chunk {i}")
            break  # Break the loop if successful
        except RateLimitError:
            print(f"Rate limit exceeded at chunk {i}. Retrying after delay...")
            retries += 1
            delay = 60 * (2 ** retries)  # Exponential backoff: 1, 2, 4, 8, 16 minutes
            print(f"Waiting for {delay} seconds before retrying...")
            time.sleep(delay)  # Wait before retrying
    if retries == max_retries:
        print(f"Max retries reached for chunk {i}. Skipping...")

Successfully processed chunk 0
Successfully processed chunk 1
Successfully processed chunk 2
Successfully processed chunk 3
Successfully processed chunk 4
Successfully processed chunk 5
Successfully processed chunk 6
Successfully processed chunk 7
Successfully processed chunk 8
Successfully processed chunk 9
Successfully processed chunk 10
Successfully processed chunk 11
Successfully processed chunk 12
Successfully processed chunk 13
Successfully processed chunk 14
Successfully processed chunk 15
Successfully processed chunk 16
Successfully processed chunk 17
Successfully processed chunk 18
Successfully processed chunk 19
Successfully processed chunk 20
Successfully processed chunk 21
Successfully processed chunk 22
Successfully processed chunk 23
Successfully processed chunk 24
Successfully processed chunk 25
Successfully processed chunk 26
Successfully processed chunk 27
Successfully processed chunk 28
Successfully processed chunk 29
Successfully processed chunk 30
Successfully proce

In [395]:
vector_store = PineconeVectorStore(pinecone_index, embed_model)
# # Example user query
# user_query = "What are the benefits of transformers in machine learning?"

# # Retrieve relevant documents
# results = retriever.get_relevant_documents(user_query)

# # Display results
# for i, result in enumerate(results):
#     print(f"Result {i + 1}:")
#     print(result.page_content)
#     print(result.metadata)

In [400]:
from langchain_openai import AzureChatOpenAI

In [401]:
llm = AzureChatOpenAI(
    model="gpt-35-turbo-16k",
    deployment_name="sayan-gpt35trubo16k",
    api_key=api_key,
    azure_endpoint=azure_endpoint[0],
    api_version=api_version
)

In [310]:
# contextualize_q_system_prompt = """Given a chat history and the latest user question \
# which might reference context in the chat history, formulate a standalone question \
# which can be understood without the chat history. Do NOT answer the question, \
# just reformulate it if needed and otherwise return it as is."""
# contextualize_q_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", contextualize_q_system_prompt),
#         MessagesPlaceholder("chat_history"),
#         ("human", "{input}"),
#     ]
# )
# history_aware_retriever = create_history_aware_retriever(
#     llm, vector_store.as_retriever(), contextualize_q_prompt
# )

In [403]:
question_prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', "You are a helpful assistant that generates follow-up questions to deepen the conversation."),
        ('system', "Based on the previous response, suggest a relevant follow-up question."),
        ('assistant', "{response}"),  # Placeholder for the assistant's response
        ('human', "Follow-up question:"),
    ]
)

# Create an LLMChain to generate questions
question_generator_chain = LLMChain(
    llm=llm,
    prompt=question_prompt_template
)

In [404]:
response_prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', "You are a knowledgeable assistant specializing in generative AI."),
        ('system', "Your responses should be structured with bullet points and detailed explanations if necessary.Only provide response if the question is related to ai"),
        ('system', "Here is the relevant information:\n{context}"),  # Placeholder for documents
        ('human', "{question}"),
    ]
)

# Combine Docs Chain - to generate responses
combine_docs_chain = load_qa_chain(
    llm=llm,
    chain_type = "stuff",
    prompt=response_prompt_template
)


In [405]:

# Create the ConversationalRetrievalChain
qa_chain = ConversationalRetrievalChain(
    retriever=vector_store.as_retriever(),
    combine_docs_chain=combine_docs_chain,
    question_generator=question_generator_chain  # Use the custom question generator chain
)


In [406]:
from langchain.schema import HumanMessage, AIMessage

In [407]:

user_query = "Discuss the ethical considerations associated with the use of generative AI."

conversation_context = []

response = qa_chain.run({"question": user_query, "chat_history": conversation_context})


conversation_context.append({"question": user_query, "answer": response})

print(response)

The use of generative AI raises several ethical considerations, including:

1. Bias and Discrimination: Generative AI systems are trained on existing data, which may contain biases and discriminatory patterns. If these biases are not addressed, the AI system may perpetuate and amplify them, leading to unfair outcomes and discrimination against certain individuals or groups.

2. Privacy and Data Protection: Generative AI systems require large amounts of data to learn and generate content. This raises concerns about the privacy and security of personal data. It is important to ensure that user data is collected and used in a transparent and ethical manner, with proper consent and protection measures in place.

3. Misinformation and Manipulation: Generative AI systems have the potential to create realistic and convincing content, including misinformation and deepfakes. This can be used to spread false information, manipulate public opinion, and deceive individuals. It is crucial to develo

In [375]:

user_query = "What are the challenges in training GANs, and how can they be addressed?"

conversation_context = []

response = qa_chain.run({"question": user_query, "chat_history": conversation_context})

conversation_context.append({"question": user_query, "answer": response})

print(response)

Training Generative Adversarial Networks (GANs) can be challenging due to several factors. Here are some of the challenges and potential solutions to address them:

1. Mode Collapse: Mode collapse occurs when the generator in a GAN fails to capture the full diversity of the data distribution and instead generates only a limited set of samples. This leads to repetitive and unrealistic outputs.

   - Addressing mode collapse can be done by using techniques like minibatch discrimination, which encourages the generator to produce diverse samples. Another approach is to use techniques like Wasserstein GAN (WGAN) or its variants, which have been shown to be more stable and less prone to mode collapse.

2. Vanishing/Exploding Gradients: GANs can suffer from vanishing or exploding gradients, which can make the training process unstable or lead to slow convergence.

   - To address vanishing gradients, techniques like weight initialization, batch normalization, and gradient clipping can be used

In [374]:
user_query = "Explain the architecture and functioning of the Transformer model in detail?"

conversation_context = []

response = qa_chain.run({"question": user_query, "chat_history": conversation_context})

conversation_context.append({"question": user_query, "answer": response})

print(response)

The Transformer is a deep learning model architecture that was introduced in the paper "Attention Is All You Need" by Vaswani et al. It is specifically designed for sequence-to-sequence tasks such as machine translation, text summarization, and language modeling. The Transformer model replaces traditional recurrent neural networks (RNNs) with self-attention mechanisms to capture dependencies between different positions in the input and output sequences. Here is a detailed explanation of the architecture and functioning of the Transformer model:

1. Encoder-Decoder Structure:
   - The Transformer model follows an encoder-decoder structure, commonly used in sequence transduction models. The encoder maps an input sequence to a continuous representation, and the decoder generates an output sequence based on this representation.
   - The encoder and decoder both consist of multiple layers. Each layer has two sub-layers: a self-attention mechanism and a feed-forward neural network.
   - The 